<a href="https://colab.research.google.com/github/HARISHKUMARPEDDINTI/Harish_fmml_labs_projects/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

1st answer

Yes, averaging the validation accuracy across multiple splits generally leads to more consistent and reliable estimates of a model’s performance. Here's why and how it works:

### **1. Reduction of Variance**
- **Single Split Issues**: Relying on a single train-validation split can result in high variance in validation accuracy because the performance might heavily depend on how the data is partitioned. Some splits might inadvertently make the task easier or harder for the model.
- **Multiple Splits Advantage**: By averaging across multiple splits, you mitigate the risk of having an unrepresentative split. This averaging smooths out the fluctuations caused by different data partitions, leading to a more stable estimate.

### **2. Improved Generalization**
- **Broader Evaluation**: Multiple splits ensure that every data point gets a chance to be in the validation set. This comprehensive evaluation provides a better understanding of how the model might perform on unseen data.
- **Overfitting Mitigation**: By testing the model across various subsets, you reduce the likelihood that the model is simply memorizing patterns specific to a single validation set.

### **3. Mitigation of Bias**
- **Balanced Assessment**: Certain splits might unintentionally introduce bias, especially if the data is not uniformly distributed. Multiple splits help average out these biases, providing a more objective performance measure.
- **Handling Data Imbalances**: When dealing with imbalanced datasets, multiple splits (especially stratified ones) ensure that each class is adequately represented in both training and validation sets across different splits.

### **4. Enhanced Robustness**
- **Consistency Across Scenarios**: Averaging across multiple splits ensures that the performance metrics are not overly dependent on any single partitioning of the data, leading to a more robust evaluation.
- **Detection of Anomalies**: If certain splits consistently yield lower or higher accuracies, it might indicate underlying issues with the data or the model that need to be addressed.

### **Common Methods Utilizing Multiple Splits**

- **k-Fold Cross-Validation**: The dataset is divided into *k* equal parts. The model is trained *k* times, each time using a different fold as the validation set and the remaining folds as the training set. The final performance metric is the average of the *k* validation accuracies.
  
- **Repeated Cross-Validation**: This involves performing k-fold cross-validation multiple times with different random splits to further ensure robustness.

- **Shuffle Split**: Randomly splits the data into training and validation sets multiple times, allowing for overlapping splits and averaging the results.

### **Considerations and Potential Drawbacks**

- **Computational Cost**: Training the model multiple times (once for each split) can be computationally intensive, especially for large datasets or complex models.
  
- **Data Leakage**: Ensure that the data splitting is done correctly to prevent information from the validation set leaking into the training process, which can artificially inflate performance metrics.
  
- **Implementation Complexity**: Managing multiple splits requires careful implementation to ensure consistency and proper averaging of results.

### **Best Practices**

1. **Use Stratified Splits for Classification**: Ensure that each split maintains the same class distribution as the overall dataset to handle imbalanced classes effectively.

2. **Consistent Preprocessing**: Apply the same data preprocessing steps across all splits to maintain consistency.

3. **Random Seed Control**: Set a random seed for reproducibility, especially when performing repeated or shuffled splits.

4. **Monitor Computational Resources**: Be mindful of the increased computational requirements and optimize your workflow accordingly, possibly by parallelizing the training process if resources allow.

### **Conclusion**

2nd answer

Yes, averaging the validation accuracy across multiple splits typically provides a more accurate and reliable estimate of a model's **test accuracy** compared to using a single train-validation split. Here's an in-depth explanation of why this is the case, along with important considerations to keep in mind:

---

## **1. Understanding Test Accuracy Estimation**

**Test Accuracy** refers to how well your trained model performs on unseen data. Since we don't have access to the actual test set during training, we rely on validation techniques to estimate this metric. The goal is to have this estimate as close as possible to the true test accuracy.

---

## **2. Single Train-Validation Split vs. Multiple Splits**

### **Single Train-Validation Split**

- **Pros:**
  - **Simplicity:** Easy to implement and computationally less intensive.
  - **Speed:** Faster since the model is trained and evaluated only once.

- **Cons:**
  - **High Variance:** The estimate can significantly vary depending on how the data is split.
  - **Potential Bias:** If the split is not representative of the overall data distribution, the estimate may be biased.
  - **Underutilization of Data:** Only a portion of the data is used for training and validation each time.

### **Multiple Splits (e.g., k-Fold Cross-Validation)**

- **Pros:**
  - **Reduced Variance:** Averaging across multiple splits smooths out fluctuations, leading to a more stable estimate.
  - **Less Bias:** Ensures that every data point has a chance to be in the validation set, making the estimate more representative.
  - **Better Data Utilization:** More of the data is used for both training and validation across different splits.

- **Cons:**
  - **Computational Overhead:** Requires training the model multiple times, which can be time-consuming.
  - **Implementation Complexity:** Managing multiple splits and aggregating results can be more complex.

---

## **3. Why Averaging Across Multiple Splits Yields a More Accurate Estimate**

### **a. Mitigation of Split-Dependent Variability**

- **Consistency Across Splits:** Different splits can lead to different validation accuracies due to varying data distributions. By averaging, you minimize the impact of any single anomalous split.
  
- **Comprehensive Evaluation:** Ensures that the model is tested against various subsets of data, capturing a wider range of scenarios and patterns.

### **b. Enhanced Generalization Insight**

- **Robust Performance Metrics:** Averaging provides a central tendency measure (like mean accuracy) that is less sensitive to outliers, offering a more reliable estimate of how the model might perform on truly unseen data.

- **Detection of Instability:** Multiple splits can reveal if the model's performance is unstable across different data subsets, indicating potential issues like overfitting.

### **c. Reduction of Overfitting Risk**

- **Balanced Training and Validation:** By exposing the model to multiple training and validation sets, the risk that the model is overfitting to a particular validation set is reduced.

---

## **4. Common Techniques Employing Multiple Splits**

### **a. k-Fold Cross-Validation**

- **Procedure:** The dataset is divided into *k* equal-sized folds. The model is trained *k* times, each time using a different fold as the validation set and the remaining *k-1* folds for training.
  
- **Benefit:** Each data point is used for both training and validation, providing a comprehensive performance evaluation.

### **b. Stratified k-Fold Cross-Validation**

- **Procedure:** Similar to k-Fold, but each fold maintains the same class distribution as the entire dataset. This is especially useful for imbalanced datasets.

- **Benefit:** Ensures that each fold is representative of the overall data distribution, leading to more accurate estimates.

### **c. Leave-One-Out Cross-Validation (LOOCV)**

- **Procedure:** Each data point serves as its own validation set, and the model is trained on all other data points.

- **Benefit:** Maximizes the training data used in each iteration, though it can be computationally expensive for large datasets.

---

## **5. Important Considerations and Caveats**

### **a. Computational Cost**

- **Impact:** Training models multiple times can be resource-intensive, especially with large datasets or complex models.

- **Mitigation:** Utilize parallel computing resources or opt for a smaller *k* in k-Fold cross-validation to balance computational load and estimation accuracy.

### **b. Data Leakage**

- **Risk:** Improper splitting can lead to information from the validation set leaking into the training process, artificially inflating validation accuracy.

- **Prevention:** Ensure strict separation between training and validation data in each split, and apply all preprocessing steps within each training-validation iteration.

### **c. Model Selection and Hyperparameter Tuning**

- **Complexity:** If cross-validation is used for both estimating test accuracy and tuning hyperparameters, there’s a risk of overfitting to the validation data.

- **Solution:** Implement nested cross-validation, where an inner loop handles hyperparameter tuning and an outer loop estimates the generalization performance.

### **d. Representativeness of Data Splits**

- **Challenge:** Even with multiple splits, if the dataset has inherent biases or is not representative of the real-world scenario, the estimate may still be inaccurate.

- **Approach:** Ensure that the dataset is as diverse and representative as possible, and consider domain-specific validation strategies if necessary.

---

## **6. Best Practices for Accurate Test Accuracy Estimation**

1. **Use Stratified Splits for Classification Tasks:**
   - Maintain class distribution across all splits to handle imbalanced datasets effectively.

2. **Implement Nested Cross-Validation for Hyperparameter Tuning:**
   - Separate the processes of model selection and performance estimation to avoid biased estimates.

3. **Ensure Proper Data Preprocessing Within Each Split:**
   - Apply scaling, encoding, and other preprocessing steps within each training-validation iteration to prevent data leakage.

4. **Maintain an Independent Test Set:**
   - After cross-validation, evaluate the final model on a completely independent test set to obtain an unbiased estimate of test accuracy.

5. **Set Random Seeds for Reproducibility:**
   - Ensure that splits are reproducible by setting random seeds, which is crucial for consistent evaluation.

6. **Monitor and Manage Computational Resources:**
   - Optimize code and utilize efficient algorithms to handle the increased computational demands of multiple splits.

---

3rd answer

Yes, increasing the number of iterations typically leads to a better estimate in iterative methods, especially in numerical simulations and algorithms like Monte Carlo simulations, optimization techniques, and machine learning training processes.

Here’s how more iterations affect the estimate:

1. **Increased Accuracy**: More iterations generally allow the algorithm to explore more of the solution space or refine the estimate. For methods that rely on randomness or approximations (e.g., Monte Carlo simulations), more iterations average out fluctuations and provide a more accurate result.

2. **Convergence**: For iterative optimization algorithms (like gradient descent), more iterations help the algorithm converge towards the optimal solution. However, after a certain point, the improvements may become very small.

3. **Reduction of Variance**: For simulations or random sampling methods, higher iterations tend to reduce the variance of the estimate, meaning the result becomes more stable and reliable over time.

4. **Diminishing Returns**: There is often a point where increasing the number of iterations leads to diminishing returns. The improvements in the estimate become smaller with each additional iteration, and the computational cost might outweigh the benefits.

5. **Overfitting (in some cases)**: In machine learning, too many iterations during training can sometimes lead to overfitting, where the model becomes too tailored to the training data and performs worse on unseen data. However, this can often be controlled with techniques like early stopping or regularization.

.

4th answer

Yes, monitoring computational resources is essential when increasing the number of iterations in resource-intensive processes. As you pointed out, optimizing the workflow and parallelizing computations can help manage the trade-off between better estimates and computational cost. Here are a few strategies to consider when monitoring and optimizing computational resources:

### 1. **Resource Utilization Monitoring**
   - **CPU/GPU Usage**: Track the usage of your CPU or GPU to ensure that you are not overloading or underutilizing resources.
   - **Memory Usage**: Keep an eye on RAM and VRAM, as memory constraints can slow down or crash processes. Use tools like `nvidia-smi` for GPU monitoring or system resource monitors like `htop`.
   - **Disk Usage and I/O**: Ensure that your disk read/write speeds are optimized and check for bottlenecks related to input/output operations.

### 2. **Parallelization and Distributed Computing**
   - **Parallel Processing**: Split your tasks across multiple CPUs or GPUs to process iterations in parallel. This can significantly reduce training or computation time. In Python, libraries like `multiprocessing` or `Dask` can be useful.
   - **Distributed Training**: For deep learning models, frameworks like TensorFlow, PyTorch, and Horovod allow distributed training across multiple nodes or GPUs, enhancing the scale of computation.
   - **Batch Processing**: In algorithms like gradient descent, using mini-batches (for stochastic gradient descent) allows for more efficient use of resources by processing smaller chunks of data in parallel.

### 3. **Optimize Code and Algorithms**
   - **Algorithm Efficiency**: Review the algorithm’s efficiency. Use vectorized operations (e.g., with NumPy, TensorFlow, or PyTorch) and avoid loops where possible.
   - **Early Stopping**: Implement early stopping criteria in optimization algorithms or machine learning training to avoid unnecessary iterations once the performance improvements diminish.
   - **Caching Intermediate Results**: In some simulations or iterative processes, caching frequently used results can avoid redundant computations, speeding up the process.
   
### 4. **Dynamic Resource Allocation**
   - **Autoscaling**: If working in cloud environments (AWS, GCP, Azure), use autoscaling to dynamically allocate resources based on demand, which can help balance performance and cost.
   - **GPU Selection**: Choose GPUs that balance your specific requirements (e.g., memory, cores) based on the workload. For example, models that require more matrix operations benefit from GPUs with more CUDA cores, while those with larger datasets benefit from higher VRAM capacity.

### 5. **Profiling and Bottleneck Identification**
   - **Code Profiling**: Use profiling tools (e.g., `cProfile` or line_profiler in Python) to identify bottlenecks in your code and optimize those sections for faster execution.
   - **Memory Profiling**: Tools like `memory_profiler` can help you understand how much memory each part of the program uses and where optimizations can be made.

### 6. **Mixed Precision Training (Deep Learning)**
   - For deep learning models, mixed precision training (i.e., using lower precision floating point formats like FP16) can reduce memory and computation requirements, speeding up training without significantly affecting the quality of the results.